In [1]:
from pyspark.sql import SparkSession
# Create Spark session
spark = SparkSession.builder.appName("MyApp").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 23:32:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/02 23:32:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("../data/retail-data/by-day/*.csv")

staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [4]:
staticSchema

StructType([StructField('InvoiceNo', StringType(), True), StructField('StockCode', StringType(), True), StructField('Description', StringType(), True), StructField('Quantity', IntegerType(), True), StructField('InvoiceDate', TimestampType(), True), StructField('UnitPrice', DoubleType(), True), StructField('CustomerID', DoubleType(), True), StructField('Country', StringType(), True)])

시계열 데이터 분석
- 데이터를 그룹화하고 집계하는 방법 필요
- 특정 고객이 대량으로 구매하는 영업 시간을 고려
- ex. 구매비용 컬럼을 추가하고 고객이 가장 많이 소비한 날을 찾음

윈도우 함수
- 집계 시 시계열 컬럼을 기준으로 각 날짜에 대한 전체 데이터를 가지는 윈도우를 구성
- 윈도우: 간격을 통해 처리 요건을 명시

스파크
- 관련 날짜의 데이터를 그룹화 함


In [ ]:
from pyspark.sql.functions import window, col

# 고객별 하루 단위 총 지출"을 계산하는 코드입니다.
# 즉, 같은 고객(CustomerId)의 구매 내역을 하루 단위로 묶어서 총 비용을 계산합니다.
# 이를 위해 window 함수를 사용하여 날짜(InvoiceDate)를 기준으로 "1일 단위"의 시간 창(Window)을 생성합니다.
# 이후, 고객별(CustomerId) 및 날짜별(InvoiceDate) 총 비용(total_cost)을 계산합니다.
# 마지막으로, 상위 5개의 결과를 출력합니다.

staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")\
  .show(5)  


+----------+--------------------+-----------------+
|CustomerId|              window|  sum(total_cost)|
+----------+--------------------+-----------------+
|   16057.0|{2011-12-05 00:00...|            -37.6|
|   14126.0|{2011-11-29 00:00...|643.6300000000001|
|   13500.0|{2011-11-16 00:00...|497.9700000000001|
|   17160.0|{2011-11-08 00:00...|516.8499999999999|
|   15608.0|{2011-11-11 00:00...|            122.4|
+----------+--------------------+-----------------+
only showing top 5 rows



In [10]:
# 로컬 모드에 적합한 셔플 파티션 수 설정: 5 (기본값: 200)

spark.conf.set("spark.sql.shuffle.partitions", "5")

In [11]:
# 스트리밍 코드
# read -> readStream
# maxFilesPerTrigger: 한 번에 읽을 파일 수 설정

streamingDataFrame = spark.readStream\
  .schema(staticSchema)\
  .option("maxFilesPerTrigger", 1)\
  .format("csv")\
  .option("header", "true")\
  .load("../data/retail-data/by-day/*.csv")

In [13]:
# 데이터프레임 스트리밍 유형인지 확인
streamingDataFrame.isStreaming

True

In [14]:
# 총 판매 금액 계산
purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")

지연 연산 -> 스트리밍 액션 호출

스트리밍 액션
- 트리거가 실행된 다음 데이터를 갱신하게 될 인메모리 테이블에 데이터를 저장한다.
- 예제. 파일마다 트리거를 실행
- 이전 집계값보다 더 큰 값이 발생한 경우에만 인메모리 테이블을 갱신한다.

In [ ]:
purchaseByCustomerPerHour.writeStream\
  .format("memory")\
  .queryName("customer_purchases")\
  .outputMode("complete")\
  .start()

In [ ]:
# purchaseByCustomerPerHour.writeStream\
#   .format("memory")\ // memory: 테이블을 메모리에 저장
#   .queryName("customer_purchases")\ // 인메모리에 저장된 테이블명
#   .outputMode("complete")\ // complete: 모든 카운트 수행 결과를 테이블에 저장
#   .start()

In [17]:
# 스트림이 시작되면 실행 결과가 어떠한 형태로 인메모리 테이블에 저장되는지 확인
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)\
  .show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|      NULL|{2011-03-29 01:00...| 33521.39999999998|
|      NULL|{2010-12-21 00:00...|31347.479999999938|
|   18102.0|{2010-12-07 00:00...|          25920.37|
|      NULL|{2010-12-10 00:00...|25399.560000000012|
|      NULL|{2010-12-17 00:00...|25371.769999999768|
+----------+--------------------+------------------+
only showing top 5 rows

